In [22]:
import tensorflow as tf
import os
import pandas as pd
import strym
from strym import strymread
import matplotlib.pyplot as plt
from pylab import rcParams
import strym.DBC_Read_Tools as dbc
import numpy as np
from strym import strymmap
import keras
from keras import layers, Model

In [23]:
data_path = '../dataset_v2.csv'
dbcfile = '../data/toyota_rav4_2020.dbc'

In [25]:
data_csv = pd.read_csv(data_path)
r =strymread(csvfile=data_path)
r.speed().describe()
# debug_num_samples = 100
# r = r.dataframe[:debug_num_samples]

,Time
count,0.0
mean,NaN
std,NaN
min,NaN
25%,NaN
50%,NaN
75%,NaN
max,NaN


In [27]:
r.speed()

,Time,Message
Clock,,


In [8]:
total_num_samples = r.size
num_train_samples = total_num_samples*9//10

features = ['Time', 'Bus', 'MessageID', 'Message', 'MessageLength']
labels = ['Time', 'Latitude', 'Longitude']
x_train_df = r[features][:]
y_train_df = r[labels][:]
x_test_df = r[features][:]
y_test_df = r[labels][:]

In [9]:
x_train = x_train_df.to_numpy()
y_train = y_train_df.to_numpy()
x_test = x_test_df.to_numpy()
y_test = y_test_df.to_numpy()

In [10]:
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

(100, 5)
(100, 3)
(100, 5)
(100, 3)


-110.824048
